In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install tensorflow-io



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 17.9 MB/s eta 0:00:00


In [11]:
import tensorflow as tf
import tensorflow_io as tfio
import numpy as np
from keras import layers
import tensorflow as tf
import os
import keras

# Constants
DATA_DIR = '/content/drive/MyDrive/video_vision_transformer/dataset_demo/'
BATCH_SIZE = 32
AUTO = tf.data.experimental.AUTOTUNE
EPOCHS = 60

# ViViT parameters
INPUT_SHAPE = (224, 224, 25, 3)  # Adjust based on your video's resolution
NUM_CLASSES = 7  # Adjust based on your classes
PROJECTION_DIM = 128
NUM_LAYERS = 8
NUM_HEADS = 8
PATCH_SIZE = (8, 8, 5)

# Optimizer
LEARNING_RATE = 1e-4


In [12]:

def process_video_file(file_path):
    """Utility function to load and preprocess a single video file."""
    video = tf.io.read_file(file_path)
    video = tfio.experimental.ffmpeg.decode_video(video)
    video = tf.image.resize(video, (224, 224))  # Resize for consistency
    video = video[:25, :, :, :]  # Limit to 25 frames
    return video / 255.0  # Normalize

def load_and_label_videos(phase, split, label):
    """Load and preprocess video files from a specific phase and split with labels."""
    phase_path = os.path.join(DATA_DIR, split, phase)
    file_paths = tf.data.Dataset.list_files(os.path.join(phase_path, '*.avi'))
    labels = tf.data.Dataset.from_tensor_slices(tf.repeat(label, len(file_paths)))
    videos = file_paths.map(process_video_file, num_parallel_calls=AUTO)
    return tf.data.Dataset.zip((videos, labels))

def prepare_datasets():
    splits = ['train', 'test', 'val']
    phases = ['1', '2', '3', '4', '5', '6', '7']

    datasets = {}
    for split in splits:
        phase_datasets = [load_and_label_videos(phase, split, i) for i, phase in enumerate(phases)]
        datasets[split] = phase_datasets[0]
        for dataset in phase_datasets[1:]:
            datasets[split] = datasets[split].concatenate(dataset)
        # Shuffle, batch, and prefetch
        if split == 'train':
            datasets[split] = datasets[split].shuffle(1000)
        datasets[split] = datasets[split].batch(BATCH_SIZE).prefetch(AUTO)

    return datasets

datasets = prepare_datasets()
train_dataset, test_dataset, val_dataset = datasets['train'], datasets['test'], datasets['val']


/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/ffmpeg_ops.py:43: UserWarning: could not load libtensorflow_io_ffmpeg_4.2.so: unable to open file: libtensorflow_io_ffmpeg_4.2.so, from paths: ['/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/libtensorflow_io_ffmpeg_4.2.so']
caused by: ['/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/libtensorflow_io_ffmpeg_4.2.so: undefined symbol: _ZN10tensorflow15TensorShapeBaseINS_11TensorShapeEEC2EN4absl12lts_202308024SpanIKlEE']
  warnings.warn(f"could not load {library}: {e}")
/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/ffmpeg_ops.py:43: UserWarning: could not load libtensorflow_io_ffmpeg_3.4.so: unable to open file: libtensorflow_io_ffmpeg_3.4.so, from paths: ['/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/libtensorflow_io_ffmpeg_3.4.so']
caused by: ['libavformat.so.57: cannot open shared object file: No such file or directory']
  warnings.warn(f"could not

NotImplementedError: in user code:

    File "<ipython-input-12-16a3c4588b0c>", line 4, in process_video_file  *
        video = tfio.experimental.ffmpeg.decode_video(video)
    File "/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/experimental/ffmpeg_ops.py", line 28, in decode_video  *
        from tensorflow_io.python.ops import (  # pylint: disable=import-outside-toplevel
    File "/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/ffmpeg_ops.py", line 48, in <module>
        _ffmpeg_ops, _decode_ops = _load_libraries(
    File "/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/ffmpeg_ops.py", line 45, in _load_libraries
        raise NotImplementedError("could not find ffmpeg after search through ", p)

    NotImplementedError: ('could not find ffmpeg after search through ', ['libtensorflow_io_ffmpeg_4.2.so', 'libtensorflow_io_ffmpeg_3.4.so', 'libtensorflow_io_ffmpeg_2.8.so'])


In [ ]:

class TubeletEmbedding(layers.Layer):
    def __init__(self, embed_dim, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.projection = layers.Conv3D(filters=embed_dim, kernel_size=patch_size, strides=patch_size, padding="VALID")
        self.flatten = layers.Reshape(target_shape=(-1, embed_dim))

    def call(self, videos):
        projected_patches = self.projection(videos)
        flattened_patches = self.flatten(projected_patches)
        return flattened_patches

class PositionalEncoder(layers.Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim

    def build(self, input_shape):
        _, num_tokens, _ = input_shape
        self.position_embedding = layers.Embedding(input_dim=num_tokens, output_dim=self.embed_dim)
        self.positions = ops.arange(0, num_tokens, 1)

    def call(self, encoded_tokens):
        encoded_positions = self.position_embedding(self.positions)
        encoded_tokens = encoded_tokens + encoded_positions
        return encoded_tokens

def create_vivit_classifier(tubelet_embedder, positional_encoder):
    inputs = layers.Input(shape=INPUT_SHAPE)
    patches = tubelet_embedder(inputs)
    encoded_patches = positional_encoder(patches)

    for _ in range(NUM_LAYERS):
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(num_heads=NUM_HEADS, key_dim=PROJECTION_DIM // NUM_HEADS, dropout=0.1)(x1, x1)
        x2 = layers.Add()([attention_output, encoded_patches])

        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = keras.Sequential([layers.Dense(units=PROJECTION_DIM * 4, activation=ops.gelu), layers.Dense(units=PROJECTION_DIM, activation=ops.gelu)])(x3)
        encoded_patches = layers.Add()([x3, x2])

    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.GlobalAvgPool1D()(representation)
    outputs = layers.Dense(units=NUM_CLASSES, activation="softmax")(representation)

    return keras.Model(inputs=inputs, outputs=outputs)

def run_experiment():
    model = create_vivit_classifier(
        tubelet_embedder=TubeletEmbedding(embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE),
        positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM)
    )

    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    _ = model.fit(train_dataset, epochs=EPOCHS, validation_data=val_dataset)

    test_loss, test_accuracy = model.evaluate(test_dataset)
    print(f"Test accuracy: {round(test_accuracy * 100, 2)}%")

    return model

model = run_experiment()
